In [30]:
import pandas as pd
import numpy as np
import csv

# Notebook 3 - Connecting building complexes and monasteries

This notebook implements the third step of the Klosterdatenbank-to-FactGrid-Workflow, which is to connect the newly created items for Building Complexes with the newly created items for religious communities.

As mentioned in Notebook 1, our data model distinguishes between the religious communities and the building complexes in which they lived and worked. In this step, both concepts are connected with each other. This is done in two directions: there is a connection from the building to the religious community ("Users" - (P1095)[https://database.factgrid.de/wiki/Property:P1095]) and a connection from the religious community to the building ("Real Estate" - (P208)[https://database.factgrid.de/wiki/Property:P208]). In the database table `gs_monastery_location`, this connection is specified in more detail. There are indications on when the building was used by the religious community. These information are represented in FactGrid with corresponding qualifiers for the relationship between the religious community and the building complex (see below).

In this notebook, the existing monasteries and building complexes in FactGrid are queried first. Subsequently, the corresponding Q-numbers are assigned to the entities from the table `gs_monastery_location`. For the specification of the qualifiers, the natural language statements from the fields `begin_note` and `end_note` are mapped to FactGrid properties using a rule-based approach. Finally, yet another import file in V1 syntax is generated.

## Preparation

In the first step, all required dataframes are loaded. In addition, two new dataframes are created by querying existing monasteries and building complexes in FactGrid. 

In [ ]:
from helper_functions import query_factgrid

# Save all required DataFrames in a Dictionary
dataframes = {}
dataframes["gs_monastery_location"] = pd.read_excel("data/exports_monasteryDB/gs_monastery_location.xlsx")
dataframes["building_complexes_in_factgrid"] = query_factgrid("building_complexes")
dataframes["monasteries_in_factgrid"] = query_factgrid("monasteries")

# Only handle data that has been created in current batch
dataframes["new_building_complexes"] = pd.read_csv("data/intermediate_results/new_building_complex_locations_ids.csv")

dataframes["gs_monastery_location"] = dataframes["gs_monastery_location"][dataframes["gs_monastery_location"]["id_monastery_location"].isin(dataframes["new_building_complexes"]["id_monastery_location"])]

# Process query results: Split result strings and convert to Integer, if necessary
dataframes["building_complexes_in_factgrid"]["item"] = dataframes["building_complexes_in_factgrid"]["item"].str.split("/").str[-1]
dataframes["building_complexes_in_factgrid"]["GSVocabTerm"] = dataframes["building_complexes_in_factgrid"]["GSVocabTerm"].str.split("Location").str[-1].astype(int)
dataframes["monasteries_in_factgrid"]["item"] = dataframes["monasteries_in_factgrid"]["item"].str.split("/").str[-1]
dataframes["monasteries_in_factgrid"]["KlosterdatenbankID"] = dataframes["monasteries_in_factgrid"]["KlosterdatenbankID"].astype(int)

In [32]:
# Merge tables together
merge = pd.merge(dataframes["gs_monastery_location"], dataframes["monasteries_in_factgrid"], how="left", left_on="gsn_id", right_on="KlosterdatenbankID").rename(columns={"item":"monastery_factgrid_id"})
merge = pd.merge(merge, dataframes["building_complexes_in_factgrid"], how="left", left_on="id_monastery_location", right_on="GSVocabTerm").rename(columns={"item":"building_factgrid_id"}).dropna(subset=["monastery_factgrid_id","building_factgrid_id"])
merge.drop(columns=["KlosterdatenbankID", "GSVocabTerm"], inplace=True)
merge

,Unnamed: 0,id_monastery_location,place_id,relocated,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,comment,longitude,latitude,location_name,main_location,diocese_id,monastery_factgrid_id,building_factgrid_id
0,26,7956,705,False,60419,1291,NaN,NaN,1525,NaN,NaN,NaN,10.419200,49.503200,Windsheim,NaN,NaN,Q1758540,Q1758523
1,541,17231,46484701,False,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,NaN,NaN,NaN,Viborg,NaN,0.0,Q1758543,Q1758524
2,924,5344,209,False,30016,1250,1260.0,um 1255,1527,NaN,NaN,NaN,9.270021,50.749630,NaN,NaN,NaN,Q1758547,Q1758525
3,982,3064,1420,False,20004,1357,NaN,NaN,1534,NaN,um 1534,NaN,8.836796,48.521692,NaN,NaN,NaN,Q1758549,Q1758526
4,2440,13102,46483121,False,8028,1409,NaN,NaN,1415,NaN,NaN,NaN,4.894167,52.369722,Amsterdam,NaN,0.0,Q1758544,Q1758527
5,2441,13103,46483121,False,8028,1415,NaN,NaN,1579,NaN,NaN,NaN,4.897115,52.371050,Amsterdam,NaN,0.0,Q1758544,Q1758528
6,3283,5599,29638,False,30279,1163,1185.0,1163/1185,1568,NaN,NaN,Bestand und Details im Hessischesn Hauptstaats...,8.140393,50.449848,NaN,NaN,NaN,Q1758542,Q1758529
7,3819,7133,6305,False,50211,1245,1295.0,vor 1295,1802,NaN,NaN,NaN,6.953623,50.944112,NaN,NaN,NaN,Q1758537,Q1758530
8,5090,3754,10109,False,20694,1345,NaN,vor 1345,1493,NaN,NaN,NaN,8.934051,48.476825,Rottenburg,NaN,NaN,Q1758545,Q1758531
9,6124,9838,46479299,False,5229,1127,NaN,NaN,1525,NaN,NaN,Daten importiert aus geplantem Lexikon der Aug...,8.575330,47.387139,NaN,NaN,NaN,Q1758536,Q1758532


## Time Qualifiers

In the monastery database, there are six fields in the table `gs_monastery_location` that define the time period during which a religious community lived and worked at a specific location. The start and end dates can be defined by a Terminus Post Quem (`location_begin_tpq`/`location_end_tpq`) and a Terminus Ante Quem (`location_begin_taq`, `location_end_taq`). Not both fields need to be filled in every time. Additionally, it is possible to add a natural language supplement to the date information using the fields `begin_note` and `end_note`.

In practice, there is a degree of uncertainty for many of the date entries in the monastery database. There are various causes and manifestations of uncertainty or vagueness. For example, in the case of the Cistercian monastery of Mariawald in Heimbach (GSN [50153](https://klosterdatenbank.adw-goe.de/gsn/50153)), two different years are listed as potential end dates for the localization of the monastery: the dissolution of the site occurred either in 1795 or 1802. In the database, the Terminus Post Quem of the end date was set to 1795 and the Terminus Ante Quem to 1802. However, this does not exactly reflect the situation, so the note field was supplemented with the entry "1795/1802". In this case, there is uncertainty due to different statements in the literature. A case of vagueness is, for example, the Benedictine abbey of Saint-Ghislain in Belgium (GSN [11681](https://klosterdatenbank.adw-goe.de/gsn/11681)). Here, it is only known that the localization in Saint-Ghislain began in the middle of the 7th century. In the relational database, this vague designation was quantified by approximating the possible time span: the Terminus Post Quem is set to 634 and the Terminus Ante Quem to 666, assuming that this time span approximately describes the "middle of the 7th century". Additionally, this is noted again in the note field.

In FactGrid, these date entries can be modeled differently. There are properties for start and end time points, as well as properties that directly correspond to the respective fields in the database: Begin date (terminus ante quem) and Begin date (terminus post quem), respectively, as well as for the end date. Additionally, data can be expressed on different levels of precision. For the case above, the value for the date could be set to `+634-01-01T00:00:00Z/7/J`, which is displayed in the interface as "7th century". With properties for further refinement of the precision of the date entries, it can also be expressed, for example, that it is an approximate date entry (Item [Q10](https://database.factgrid.de/wiki/Item:Q10) - "circa").

To reflect the information present in the monastery database in the best possible way, each date specification in the table `gs_monastery_location` is treated in differnt steps: Firstly, if there is no note on it, the Terminus Post Quem ist considered to be the most correct date, hence the field `[begin/end]_date_tpq` is mapped to either (P49)[https://database.factgrid.de/wiki/Property:P49] (Begin date) or (P50)[https://database.factgrid.de/wiki/Property:P50] (End date). For the cases in which there is a note on the date, it is generally assumed that this note reflects the dating circumstances better than the field for terminus post and ante quem. Therefore the note is parsed using a range of regular expressions. The details on this can be found in the `helper_function.py` file, where the method is written and documented. During the process, the precision of any date is set zu "century" if indicated within the date note. Lastly, all remaining fields are filled with the value from the respective Terminus Post Quem field. These are the cases, in which the parsing of the note did not return any sufficient results. 

In general, it has to be acknowledged that the dates that are provided within the monastery database dataset are historical dates that naturally come with a degree of vagueness. The quantification of these dates, or in other words, fitting the dates into an existing datamodel, adds another layer of potential uncertainty that has to be taken into consideration when working with the dataset.

To begin with, the qualifiers are processed and added to the table that links monasteries to building complexes.

In [33]:
from helper_functions import process_date_parsing_results, parse_date, DateType

# Create a new dataframe for import and fill it with monastery-building complex pairs
monastery_to_building = pd.DataFrame()
monastery_to_building["qid"] = merge["monastery_factgrid_id"]
monastery_to_building["P208"] = merge["building_factgrid_id"]

# Getting relevant columns from the dataframe above
monastery_to_building["location_begin_tpq"] = merge["location_begin_tpq"]
monastery_to_building["location_end_tpq"] = merge["location_end_tpq"]
monastery_to_building["location_begin_note"] = merge["location_begin_note"]
monastery_to_building["location_end_note"] = merge["location_end_note"]

# Parse date notes
monastery_to_building['begin_date_parse_result'] = merge["location_begin_note"].apply(lambda x: parse_date(str(x), DateType.BEGIN_DATE))
monastery_to_building['end_date_parse_result'] = merge["location_end_note"].apply(lambda x: parse_date(str(x), DateType.END_DATE))

# Add date notes as Qualifiers 787 and 788
monastery_to_building['qal787'] = merge["location_begin_note"].apply(lambda x: f"\"{x}\"" if not pd.isna(x) else np.nan)
monastery_to_building['qal788'] = merge["location_end_note"].apply(lambda x: x if x != "heute" else np.nan).apply(lambda x: f"\"{x}\"" if not pd.isna(x) else np.nan)

# Add more Qualifiers as indicated by date parsing results
process_date_parsing_results(monastery_to_building, "location")

# Cleanup & Add Source Statements
monastery_to_building = monastery_to_building.drop(columns=["location_begin_tpq", "location_end_tpq", "begin_date_parse_result", "end_date_parse_result"])
monastery_to_building["S471"] = merge["gsn_id"].apply(lambda x: f"\"{x}\"" if not pd.isna(x) else np.nan)
monastery_to_building.drop(columns={"location_begin_note", "location_end_note"}, inplace=True)
monastery_to_building

,qid,P208,qal787,qal788,qal49,qal50,qal785,qal786,qal1124,qal1126,S471
0,Q1758540,Q1758523,NaN,NaN,+1291-01-01T00:00:00Z/9/J,+1525-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""60419"""
1,Q1758543,Q1758524,"""1285 erstmals erwähnt""","""1526 noch existent""",+1285-00-00T00:00:00Z/9/J,+1526-00-00T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""11787"""
2,Q1758547,Q1758525,"""um 1255""",NaN,+1255-00-00T00:00:00Z/9/J,+1527-01-01T00:00:00Z/9/J,Q10,NaN,NaN,NaN,"""30016"""
3,Q1758549,Q1758526,NaN,"""um 1534""",+1357-01-01T00:00:00Z/9/J,+1534-00-00T00:00:00Z/9/J,NaN,Q10,NaN,NaN,"""20004"""
4,Q1758544,Q1758527,NaN,NaN,+1409-01-01T00:00:00Z/9/J,+1415-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""8028"""
5,Q1758544,Q1758528,NaN,NaN,+1415-01-01T00:00:00Z/9/J,+1579-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""8028"""
6,Q1758542,Q1758529,"""1163/1185""",NaN,+1163-00-00T00:00:00Z/9/J,+1568-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""30279"""
7,Q1758537,Q1758530,"""vor 1295""",NaN,NaN,+1802-01-01T00:00:00Z/9,NaN,NaN,+1295-00-00T00:00:00Z/9/J,NaN,"""50211"""
8,Q1758545,Q1758531,"""vor 1345""",NaN,NaN,+1493-01-01T00:00:00Z/9/J,NaN,NaN,+1345-00-00T00:00:00Z/9/J,NaN,"""20694"""
9,Q1758536,Q1758532,NaN,NaN,+1127-01-01T00:00:00Z/9/J,+1525-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""5229"""


The cell below saves the result to the supported export formats

In [34]:
from helper_functions import df_to_qs_v1
monastery_to_building.to_excel("data/results/monastery_building_connection/monastery_to_building.xlsx", index=False)
monastery_to_building.to_csv("data/results/monastery_building_connection/monastery_to_building.csv", index=False, doublequote=False, quoting=csv.QUOTE_NONE, escapechar="§")
with open("data/results/monastery_building_connection/monastery_to_building.tsv", "w") as file:
    file.write(df_to_qs_v1(monastery_to_building))
monastery_to_building

,qid,P208,qal787,qal788,qal49,qal50,qal785,qal786,qal1124,qal1126,S471
0,Q1758540,Q1758523,NaN,NaN,+1291-01-01T00:00:00Z/9/J,+1525-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""60419"""
1,Q1758543,Q1758524,"""1285 erstmals erwähnt""","""1526 noch existent""",+1285-00-00T00:00:00Z/9/J,+1526-00-00T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""11787"""
2,Q1758547,Q1758525,"""um 1255""",NaN,+1255-00-00T00:00:00Z/9/J,+1527-01-01T00:00:00Z/9/J,Q10,NaN,NaN,NaN,"""30016"""
3,Q1758549,Q1758526,NaN,"""um 1534""",+1357-01-01T00:00:00Z/9/J,+1534-00-00T00:00:00Z/9/J,NaN,Q10,NaN,NaN,"""20004"""
4,Q1758544,Q1758527,NaN,NaN,+1409-01-01T00:00:00Z/9/J,+1415-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""8028"""
5,Q1758544,Q1758528,NaN,NaN,+1415-01-01T00:00:00Z/9/J,+1579-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""8028"""
6,Q1758542,Q1758529,"""1163/1185""",NaN,+1163-00-00T00:00:00Z/9/J,+1568-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""30279"""
7,Q1758537,Q1758530,"""vor 1295""",NaN,NaN,+1802-01-01T00:00:00Z/9,NaN,NaN,+1295-00-00T00:00:00Z/9/J,NaN,"""50211"""
8,Q1758545,Q1758531,"""vor 1345""",NaN,NaN,+1493-01-01T00:00:00Z/9/J,NaN,NaN,+1345-00-00T00:00:00Z/9/J,NaN,"""20694"""
9,Q1758536,Q1758532,NaN,NaN,+1127-01-01T00:00:00Z/9/J,+1525-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""5229"""


For the other direction, from building to monastery, we can simply swap the columns qid and P208 and change the Property Name to P1095

In [35]:
p1095 = monastery_to_building["qid"]
building_to_monastery = monastery_to_building.copy()
building_to_monastery["qid"] = monastery_to_building["P208"]
building_to_monastery["P208"] = p1095
building_to_monastery.rename(columns={"P208":"P1095"}, inplace=True)
building_to_monastery

,qid,P1095,qal787,qal788,qal49,qal50,qal785,qal786,qal1124,qal1126,S471
0,Q1758523,Q1758540,NaN,NaN,+1291-01-01T00:00:00Z/9/J,+1525-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""60419"""
1,Q1758524,Q1758543,"""1285 erstmals erwähnt""","""1526 noch existent""",+1285-00-00T00:00:00Z/9/J,+1526-00-00T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""11787"""
2,Q1758525,Q1758547,"""um 1255""",NaN,+1255-00-00T00:00:00Z/9/J,+1527-01-01T00:00:00Z/9/J,Q10,NaN,NaN,NaN,"""30016"""
3,Q1758526,Q1758549,NaN,"""um 1534""",+1357-01-01T00:00:00Z/9/J,+1534-00-00T00:00:00Z/9/J,NaN,Q10,NaN,NaN,"""20004"""
4,Q1758527,Q1758544,NaN,NaN,+1409-01-01T00:00:00Z/9/J,+1415-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""8028"""
5,Q1758528,Q1758544,NaN,NaN,+1415-01-01T00:00:00Z/9/J,+1579-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""8028"""
6,Q1758529,Q1758542,"""1163/1185""",NaN,+1163-00-00T00:00:00Z/9/J,+1568-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""30279"""
7,Q1758530,Q1758537,"""vor 1295""",NaN,NaN,+1802-01-01T00:00:00Z/9,NaN,NaN,+1295-00-00T00:00:00Z/9/J,NaN,"""50211"""
8,Q1758531,Q1758545,"""vor 1345""",NaN,NaN,+1493-01-01T00:00:00Z/9/J,NaN,NaN,+1345-00-00T00:00:00Z/9/J,NaN,"""20694"""
9,Q1758532,Q1758536,NaN,NaN,+1127-01-01T00:00:00Z/9/J,+1525-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""5229"""


Finally, save the file in the available export formats

In [36]:
building_to_monastery.to_excel("data/results/monastery_building_connection/building_to_monastery.xlsx", index=False)
building_to_monastery.to_csv("data/results/monastery_building_connection/building_to_monastery.csv", index=False, doublequote=False, quoting=csv.QUOTE_NONE, escapechar="§")
with open("data/results/monastery_building_connection/building_to_monastery.tsv", "w") as file:
    file.write(df_to_qs_v1(building_to_monastery))
building_to_monastery

,qid,P1095,qal787,qal788,qal49,qal50,qal785,qal786,qal1124,qal1126,S471
0,Q1758523,Q1758540,NaN,NaN,+1291-01-01T00:00:00Z/9/J,+1525-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""60419"""
1,Q1758524,Q1758543,"""1285 erstmals erwähnt""","""1526 noch existent""",+1285-00-00T00:00:00Z/9/J,+1526-00-00T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""11787"""
2,Q1758525,Q1758547,"""um 1255""",NaN,+1255-00-00T00:00:00Z/9/J,+1527-01-01T00:00:00Z/9/J,Q10,NaN,NaN,NaN,"""30016"""
3,Q1758526,Q1758549,NaN,"""um 1534""",+1357-01-01T00:00:00Z/9/J,+1534-00-00T00:00:00Z/9/J,NaN,Q10,NaN,NaN,"""20004"""
4,Q1758527,Q1758544,NaN,NaN,+1409-01-01T00:00:00Z/9/J,+1415-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""8028"""
5,Q1758528,Q1758544,NaN,NaN,+1415-01-01T00:00:00Z/9/J,+1579-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""8028"""
6,Q1758529,Q1758542,"""1163/1185""",NaN,+1163-00-00T00:00:00Z/9/J,+1568-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""30279"""
7,Q1758530,Q1758537,"""vor 1295""",NaN,NaN,+1802-01-01T00:00:00Z/9,NaN,NaN,+1295-00-00T00:00:00Z/9/J,NaN,"""50211"""
8,Q1758531,Q1758545,"""vor 1345""",NaN,NaN,+1493-01-01T00:00:00Z/9/J,NaN,NaN,+1345-00-00T00:00:00Z/9/J,NaN,"""20694"""
9,Q1758532,Q1758536,NaN,NaN,+1127-01-01T00:00:00Z/9/J,+1525-01-01T00:00:00Z/9/J,NaN,NaN,NaN,NaN,"""5229"""
